In [1]:
# import the necessary libraries
import pathlib
import numpy as np
import os
import time
#import tensorflow as tf

In [2]:
import tensorflow as tf
import os
print(tf.__version__)
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2021-09-18 18:55:46.805276: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


2.4.1
Num GPUs Available:  1


2021-09-18 18:55:48.781181: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-18 18:55:48.801679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-18 18:55:48.801949: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:02.0 name: GRID RTX8000-2Q computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 2.00GiB deviceMemoryBandwidth: 625.94GiB/s
2021-09-18 18:55:48.801970: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-09-18 18:55:48.848917: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-09-18 18:55:48.848978: I tensorflow/stream_executor/platform/def

In [3]:
# # Enable or disable GPU
# ENABLE_GPU = False
# tf.debugging.set_log_device_placement(True)
# print("TensorFlow version: ", tf.__version__)
# devices = tf.config.list_physical_devices(device_type=None)
# if not ENABLE_GPU:
#     tf.config.set_visible_devices([], 'GPU')
# device_name = tf.test.gpu_device_name()
# if not device_name:
#     print('Running on CPU')
# else:
#     print('Found GPU at: {}'.format(device_name))

In [4]:
#define path to the dataset
path = pathlib.Path('/local/tiny-imagenet-200')

In [5]:
# First load wnids
with open(os.path.join(path, 'wnids.txt'), 'r') as f:
  wnids = [x.strip() for x in f]

# Map wnids to integer labels
wnid_to_label = {wnid: i for i, wnid in enumerate(wnids)}
label_to_wnid = {v: k for k, v in wnid_to_label.items()}


# Use words.txt to get names for each class
with open(os.path.join(path, 'words.txt'), 'r') as f:
  wnid_to_words = dict(line.split('\t') for line in f)
  for wnid, words in wnid_to_words.items():
      wnid_to_words[wnid] = [w.strip() for w in words.split(',')]
class_names = [wnid_to_words[wnid] for wnid in wnids]




In [6]:
# Function to load training and validation data

# Import necessary packages
from __future__ import print_function
from builtins import range
from six.moves import cPickle as pickle
import numpy as np
import os
import platform
from tqdm import tqdm
import random
from imageio import imread
from imageio import imsave




# Function for loading the tiny imagenet data
def load_tiny_imagenet(path, is_training=True, dtype=np.float32, subtract_mean=True, debug=False, debug_nclass=3):
    """
    Load TinyImageNet. Each of TinyImageNet-100-A, TinyImageNet-100-B, and
    TinyImageNet-200 have the same directory structure, so this can be used
    to load any of them.

    Note: The original implementation loaded data as NCHW, I (tyler) changed it to NHWC

    Inputs:
    - path: String giving path to the directory to load.
    - is_training: If True, dont load testing data, if False, dont load training and val data
        Note: Must always load training data in order to subtract_mean.
    - dtype: numpy datatype used to load the data.
    - subtract_mean: Whether to subtract the mean training image.
    - debug: Whether or not to load a small number of classes for debugging

    Returns: A dictionary with the following entries:
    - class_names: A list where class_names[i] is a list of strings giving the
      WordNet names for class i in the loaded dataset.
    - X_train: (N_tr, 64, 64, 3) array of training images
    - y_train: (N_tr,) array of training labels
    - X_val: (N_val, 64, 64, 3) array of validation images
    - y_val: (N_val,) array of validation labels
    - X_test: (N_test, 64, 64, 3) array of testing images.
    - y_test: (N_test,) array of test labels; if test labels are not available
      (such as in student code) then y_test will be None.
    - mean_image: (64, 64, 3) array giving mean training image
    - label_to_wnid: dictionary with mapping from integer class label to wnid
    """
    # First load wnids
    with open(os.path.join(path, 'wnids.txt'), 'r') as f:
        wnids = [x.strip() for x in f]

    # Map wnids to integer labels
    wnid_to_label = {wnid: i for i, wnid in enumerate(wnids)}
    label_to_wnid = {v: k for k, v in wnid_to_label.items()}

    # Use words.txt to get names for each class
    with open(os.path.join(path, 'words.txt'), 'r') as f:
        wnid_to_words = dict(line.split('\t') for line in f)
        for wnid, words in wnid_to_words.items():
            wnid_to_words[wnid] = [w.strip() for w in words.split(',')]
    class_names = [wnid_to_words[wnid] for wnid in wnids]

    if debug:
        print('Debug is on! Only loading %d / %d training classes.'
                  % (debug_nclass, len(wnids)))

    # Next load training data.
    X_train, y_train = [], []
    train_wnids = wnids[:debug_nclass] if debug else wnids
    for i, wnid in tqdm(enumerate(train_wnids), total=len(train_wnids)):
        # To figure out the filenames we need to open the boxes file
        boxes_file = os.path.join(path, 'train', wnid, '%s_boxes.txt' % wnid)
        with open(boxes_file, 'r') as f:
            filenames = [x.split('\t')[0] for x in f]
        num_images = len(filenames)

        X_train_block = np.zeros((num_images, 64, 64, 3), dtype=dtype)
        y_train_block = wnid_to_label[wnid] * \
                        np.ones(num_images, dtype=np.int64)
        for j, img_file in enumerate(filenames):
            img_file = os.path.join(path, 'train', wnid, 'images', img_file)
            img = imread(img_file)
            if img.ndim == 2:   ## grayscale file
                img.shape = (64, 64, 1)
            X_train_block[j] = img
        X_train.append(X_train_block)
        y_train.append(y_train_block)

    # We need to concatenate all training data
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)

    # Next load validation data
    X_val, y_val = None, None
    if is_training:
        print('loading validation data')
        with open(os.path.join(path, 'val', 'val_annotations.txt'), 'r') as f:
            img_files = []
            val_wnids = []
            for line in f:
                img_file, wnid = line.split('\t')[:2]
                img_files.append(img_file)
                val_wnids.append(wnid)
            num_val = len(img_files)
            y_val = np.array([wnid_to_label[wnid] for wnid in val_wnids])
            X_val = np.zeros((num_val, 64, 64, 3), dtype=dtype)
            for i, img_file in tqdm(enumerate(img_files), total=len(img_files)):
                img_file = os.path.join(path, 'val', 'images', img_file)
                img = imread(img_file)
                if img.ndim == 2:
                    img.shape = (64, 64, 1)
                X_val[i] = img

    mean_image = None
    if subtract_mean:
        mean_image = X_train.mean(axis=0)
        if is_training:
            X_train -= mean_image[None]
            X_val -= mean_image[None]
        else:
            X_test -= mean_image[None]

    if not is_training:
        X_train = None
        y_train = None

    return {
      'class_names': class_names,
      'X_train': X_train,
      'y_train': y_train,
      'X_val': X_val,
      'y_val': y_val,
      'mean_image': mean_image,
      'label_to_wnid': label_to_wnid
    }


In [7]:
print ("Loading Tiny-Imagenet Dataset for training and validation data")
dataset_train_val = load_tiny_imagenet(path, is_training = True, dtype=np.float32, subtract_mean=False)
x_train = dataset_train_val["X_train"] # 100000 images when each is a 64*64*3
y_train = dataset_train_val["y_train"] # 100000 elements
#slice those suckers down
x_train = x_train[1:50000]
y_train = y_train[1:50000]
x_val = dataset_train_val["X_val"] # 10000 images when each is a 64*64*3
y_val = dataset_train_val["y_val"] # 10000 elements



Loading Tiny-Imagenet Dataset for training and validation data


100%|█████████████████████████████████████████| 200/200 [01:08<00:00,  2.94it/s]


loading validation data


100%|███████████████████████████████████| 10000/10000 [00:07<00:00, 1325.94it/s]


In [8]:
x_train=x_train/255.0
x_val=x_val/255.0

In [9]:
# # one hot encode y data
# from tensorflow.keras.utils import to_categorical
# num_classes = 200
# y_train = to_categorical(y_train, num_classes)
# y_val = to_categorical(y_val, num_classes)

In [10]:
# Import required packages for training
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, ZeroPadding2D,Convolution2D, Activation, Dropout 
from tensorflow.keras import datasets


#from __future__ import absolute_import, division, print_function, unicode_literals

In [11]:
# Basic CNN model
model = Sequential()

# conv1
model.add(Conv2D(32, (5, 5), input_shape= (64, 64, 3),activation='relu'))
model.add(Conv2D(32,(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

# fc1
model.add(Dense(256,activation='relu'))

# fc2
model.add(Dense(200,activation='softmax'))

# sgd = SGD(lr=0.01, momentum=0.9)
#model.compile(loss='categorical_crossentropy' ,optimizer='adam',metrics=['accuracy'])


#model.compile(loss='sparse_categorical_crossentropy' ,optimizer='adam',metrics=['accuracy'])



model.compile(
    optimizer=tf.keras.optimizers.SGD(),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
            tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='Top5'),
            tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='Top1')],
)

#model.summary()


2021-09-18 18:57:08.389135: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-18 18:57:08.398173: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294605000 Hz
2021-09-18 18:57:08.398869: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ea9765c6b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-18 18:57:08.398893: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-18 18:57:08.399204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so

In [ ]:
# Training on the tiny image dataset
# Experiment here with the batch size, number of epochs etc as mentioned in the lab1 doc.
# model.fit(x_train,y_train, validation_data=(x_val, y_val),batch_size=256,callbacks=callbacks,
#           epochs=100)


# model.fit(x_train,y_train, validation_data=(x_val, y_val),batch_size=32,
#           epochs=15)

#model.fit_generator(x_train,y_train, validation_data=(x_val, y_val),batch_size=32,epochs=10)
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)
start_time = time.time()

# model.fit(x_train,y_train, validation_split=0.3,batch_size=32,
#           epochs=15)
history = model.fit(x_train,y_train, validation_data=(x_val, y_val),batch_size=32,
          epochs=20)


print("Time taken: %.2fs" % (time.time() - start_time))

2021-09-18 18:57:10.068709: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2457550848 exceeds 10% of free system memory.
2021-09-18 18:57:12.005426: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/20


2021-09-18 18:57:12.592799: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-09-18 18:57:13.116745: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-09-18 18:57:15.803189: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-09-18 18:57:15.916202: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-09-18 18:57:17.442917: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 370.77M (388784128 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2021-09-18 18:57:17.443058: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 333.70M (349905920 bytes) from device:

1555/1563 [============================>.] - ETA: 0s - loss: 4.9728 - sparse_categorical_accuracy: 0.0095 - Top5: 0.0513 - Top1: 0.0095

2021-09-18 18:57:43.662208: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 386.77M (405561344 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2021-09-18 18:57:43.662260: W tensorflow/core/common_runtime/bfc_allocator.cc:314] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2021-09-18 18:57:43.663728: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 450.77M (472670208 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2021-09-18 18:57:43.663880: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to al

In [ ]:
# Save the cnn model
model.save('CNN_TinyImageNet_trained.h5')

In [ ]:
model.evaluate(x=x_val, y=y_val)

In [ ]:
#### 4-1
###Batch 32
## Top5: 0.4535
## Top1: 0.2237

#### 4-3
## idk if this is right but for 1/4 dataset
## Top5: 0.1321
## Top1: 0.0586
## 1/2 dataset
## Top5: 0.0358
## Top1: 0.0077

In [ ]:
#4-2
#instead of 3 I just plotted up to 40
#a the validation accuracy increased up until a point where it starts to overtrain on the
#b Since I was only able to run for 40 epochs since the Jupyter notebooks keep crashing I've spent hours on this already
# Top5 0.3596999943256378,
# Top1 0.15940000116825104
# was the end
# somewhere in the middle it peaks around 0.45 for top 5 and 0.224 for top 1
#c Like I said in a, the reason the validation accuracy starts going down is you overtrain/overfit for the training dataset so your model
# is not generalized enough to work on new inputs (validation data)
from matplotlib import pyplot as plt

plt.plot(history.history['val_Top1'])
plt.plot(history.history['val_Top5'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Top1', 'Top5'], loc='upper left')
plt.show()


In [ ]:
# Place the logs in a timestamped subdirectory
# This allows to easy select different training runs
# In order not to overwrite some data, it is useful to have a name with a timestamp
#log_dir="C:/Temp/CPRE482X/Lab1/Files/Training/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Specify the callback object
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# tf.keras.callback.TensorBoard ensures that logs are created and stored
# We need to pass callback object to the fit method
# The way to do this is by passing the list of callback objects, which is in our case just one

In [ ]:
# # Using early stopping to monitor validation accuracy
# callbacks = [
#     tf.keras.callbacks.EarlyStopping(
#         # Stop training when `val_loss` is no longer improving
#         monitor="val_loss",
#         # "no longer improving" being defined as "no better than 1e-2 less"
#         min_delta=1e-2,
#         # "no longer improving" being further defined as "for at least 2 epochs"
#         patience=2,
#         verbose=1,
#     )
# #     tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    
# ]